# Using fhir-pipe

Here is a demo of our pipe which takes in input **mapping rules** and a **SQL database** and output data in the **FHIR format**.

## Set up

Before starting, you need... mapping and a SQL database! Fortunately we have already provided some rules in the repository [`fhir-mapping`](https://github.com/arkhn/fhir-mapping), so all you have to do is to clone it in the same folder than `fhir-pipe`. Regarding data, well, we have provided a small SQL script `config_cw_local.sql` which will build a very small and fake database following the CW format and which is for illustrative purpose. To build your mock `cw_local` database, [install psql](https://www.postgresql.org/download/) (_you only need the command line tool_) and run:

    psql -f config_cw_local.sql 

## Let's get started !

In [1]:
import json
import arkhn

> If it doesn't work, import it in your bash_profile / bash_rc if necessary, eg you can add the line as we don't have a `setup.py` so far (<-- Good First Issue)
>
>     export PYTHONPATH=$PYTHONPATH:/Users/ryffel/Documents/Code/fhir_pipe

Precise the project and the FHIR resource you want to fill

In [2]:
project = 'CW'
resource = 'patient'
data = arkhn.loader.load(project, resource)

In [3]:
patient = data['fhir']
info = data['info']

### Fetch the data from SQL

Build the SQL query, and output also the graph of joins

In [4]:
sql_query, squash_rules, graph = arkhn.parser.build_sql_query(project, patient, info)

PATIENT --- PATCOMP
PATIENT --- PATADR
PATADR --- PAYS
PATIENT -<= PAT_PAP
PAT_PAP --- TYPELIEN


In [5]:
graph.get('PATIENT')

[PATIENT O2O:(PATCOMP,PATADR) O2M:(PAT_PAP)]

In [6]:
print(squash_rules)
print(sql_query)

[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16), [[(17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28), []]]]
SELECT PATCOMP.NUMSECU, PATCOMP.INSEE, PATIENT.NOMPAT, PATIENT.PREPAT, PATADR.TEL, PATIENT.SEXE, PATIENT.DTNAIS, PATIENT.DECEDE, PATIENT.DTDECES, PATADR.ADR1, PATADR.ADR2, PATADR.ADR3, PATADR.ADR4, PATADR.VILLE, PATADR.CP, PAYS.LIBELLE, PATCOMP.SITUAFAM, TYPELIEN.LIENFAMILIAL, PAT_PAP.NOM, PAT_PAP.PRENOM, PAT_PAP.TELDOM, PAT_PAP.TELDOM, PAT_PAP.TELDOM, PAT_PAP.TELMOB, PAT_PAP.TELMOB, PAT_PAP.TELMOB, PAT_PAP.TELPROF, PAT_PAP.TELPROF, PAT_PAP.TELPROF FROM PATIENT LEFT JOIN PATCOMP ON PATIENT.NOPAT = PATCOMP.NOPAT LEFT JOIN PATADR ON PATIENT.NOPAT = PATADR.NOPAT LEFT JOIN PAYS ON PATADR.NOPAYS = PAYS.NOPAYS LEFT JOIN PAT_PAP ON PATIENT.NOPAT = PAT_PAP.NOPAT LEFT JOIN TYPELIEN ON PAT_PAP.NOTYPE_LP = TYPELIEN.NOTYPELIEN;


Run the SQL query

In [7]:
rows = arkhn.sql.run(sql_query)

Show results 

In [8]:
for row in rows:
    print(row)

('188097728809897', '27770', 'SARDOU', 'MICHELLE', '01-34.87 21 00', 'F', '19700401', 'N', '', '3 rue Soufflot', 'Batiment 23', '', '', 'Paris 5e', '75005', 'FRANCE', 'M', 'Pere', 'GOTTAR', 'Michel', '0666775328', '0666775328', '0666775328', '', '', '', '', '', '')
('NaN', '262047511447626', 'KIRIKOU', 'MARCEL', '01-00.87 20 00', 'M', '19940506', 'O', '20180922', '69 Bvd St Germain', 'Hôtel de Guermantes', '', '', 'Paris', '75006', 'LIECHTENSTEIN', 'V', 'Mere', 'Guermantes', 'Alphonse', '06 86 79 53 63', '06 86 79 53 63', '06 86 79 53 63', '', '', '', '', '', '')
('NaN', '262047511447626', 'KIRIKOU', 'MARCEL', '01-00.87 20 00', 'M', '19940506', 'O', '20180922', '69 Bvd St Germain', 'Hôtel de Guermantes', '', '', 'Paris', '75006', 'LIECHTENSTEIN', 'V', 'Femme', 'De Saint Loup', 'Robert', '', '', '', '07 86 29 00 61', '07 86 29 00 61', '07 86 29 00 61', '', '', '')


### Process the data and fill in the FHIR format

Apply the One to Many joins: for example here rows 2 and 3 refer to the same patient, but with two contact person to call. Using `squash_rules` you can merge/squash these two rows.

In [9]:
rows = arkhn.sql.apply_joins(rows, squash_rules)

There is now only 2 rows

In [10]:
for row in rows:
    print(row)

['188097728809897', '27770', 'SARDOU', 'MICHELLE', '01-34.87 21 00', 'F', '19700401', 'N', '', '3 rue Soufflot', 'Batiment 23', '', '', 'Paris 5e', '75005', 'FRANCE', 'M', [['Pere', 'GOTTAR', 'Michel', '0666775328', '0666775328', '0666775328', '', '', '', '', '', '']]]
['NaN', '262047511447626', 'KIRIKOU', 'MARCEL', '01-00.87 20 00', 'M', '19940506', 'O', '20180922', '69 Bvd St Germain', 'Hôtel de Guermantes', '', '', 'Paris', '75006', 'LIECHTENSTEIN', 'V', [['Mere', 'Guermantes', 'Alphonse', '06 86 79 53 63', '06 86 79 53 63', '06 86 79 53 63', '', '', '', '', '', ''], ['Femme', 'De Saint Loup', 'Robert', '', '', '', '07 86 29 00 61', '07 86 29 00 61', '07 86 29 00 61', '', '', '']]]


Generate JSON and output it to `data.txt`

In [11]:
json_rows = []
for row in rows:
    print('***')
    row = list(row)
    tree = arkhn.parser.dfs_create_fhir(project, patient, row)
    tree, n_leafs = arkhn.parser.clean_fhir(tree)
    json_rows.append(tree)
    print('Entity with', n_leafs, 'elems')
    print(json.dumps(tree, indent=2, ensure_ascii=False))

# Uncomment to write to file
# arkhn.parser.write_to_file(json_rows, 'integrated_data/data.txt')

***
Entity with 22 elems
{
  "resourceType": "Patient",
  "identifier": [
    {
      "type": {
        "text": "Num secu INSEE"
      },
      "value": "188097728809897"
    }
  ],
  "name": [
    {
      "family": "Sardou",
      "given": [
        "Michelle"
      ]
    }
  ],
  "telecom": [
    {
      "system": "phone",
      "value": "01 34 87 21 00"
    }
  ],
  "gender": "female",
  "birthDate": "1970-04-01T00:00:00",
  "deceasedBoolean": false,
  "address": [
    {
      "line": [
        "3 rue Soufflot",
        "Batiment 23"
      ],
      "city": "Paris 5e",
      "postalCode": "75005",
      "country": "France"
    }
  ],
  "maritalStatus": {
    "text": "Marié(e)"
  },
  "contact": [
    {
      "relationship": [
        {
          "text": "Pere"
        }
      ],
      "name": {
        "family": "Gottar",
        "given": [
          "Michel"
        ]
      },
      "telecom": [
        {
          "system": "phone",
          "value": "06 66 77 53 28",
          "u

## Import mappings from distant end point

In [1]:
import requests

In [2]:
def run_query(endpoint, headers, query):
    request = requests.post(
        endpoint,
        headers=headers,
        json={
            'query': query,
            'variables': None,
            'operationName': None,
        }
    )

    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed with returning code {}.".format(request.status_code))

In [3]:
endpoint = 'https://graphql.live.arkhn.org/'

In [4]:
headers = {
    "content-type": "application/json",
}

In [18]:
query = """
fragment entireJoin on Join {{
    id
    sourceOwner
    sourceTable
    sourceColumn
    targetOwner
    targetTable
    targetColumn
}}

fragment entireInputColumn on InputColumn {{
    id
    owner
    table
    column
    script
    staticValue
    joins {{
        ...entireJoin
    }}
}}

fragment a on Attribute {{
    id
    comment
    name
    mergingScript
    isProfile
    type
    inputColumns {{
        ...entireInputColumn
    }}
}}

query {{
    getResource (database: "{0}", resource: "{1}") {{
        id
        name
        attributes {{
            ...a
            attributes {{
                ...a
                attributes {{
                    ...a
                    attributes {{
                        ...a
                        attributes {{
                            ...a
                            attributes {{
                                ...a
                                attributes {{
                                    ...a
                                    attributes {{
                                        ...a
                                        attributes {{
                                            ...a
                                            attributes {{
                                                ...a
                                                attributes {{
                                                    ...a
                                                    attributes {{
                                                        ...a
                                                        attributes {{
                                                            ...a
                                                            attributes {{
                                                                ...a
                                                                attributes {{
                                                                    ...a
                                                                }}
                                                            }}
                                                        }}
                                                    }}
                                                }}
                                            }}
                                        }}
                                    }}
                                }}
                            }}
                        }}
                    }}
                }}
            }}
        }}
    }}
}}
"""

In [19]:
def get_fhir_resource(database, resource):
    return run_query(endpoint, headers, query.format(database, resource))

In [20]:
print(run_query(endpoint, headers, query.format('Crossway', 'Patient')))

{'data': {'getResource': {'id': 'cjpicvbkxusn60a57glvgvc90', 'name': 'Patient', 'attributes': [{'id': 'cjpicvbl2usn70a57x5juy08k', 'comment': 'An identifier for this patient', 'name': 'identifier', 'mergingScript': None, 'isProfile': None, 'type': 'list::Identifier', 'inputColumns': [], 'attributes': [{'id': 'cjpicvbl5usn90a57dhmj6m07', 'comment': None, 'name': 'Identifier_0', 'mergingScript': None, 'isProfile': True, 'type': 'Identifier', 'inputColumns': [], 'attributes': [{'id': 'cjpicvbl9usnb0a5759d0bixj', 'comment': 'usual | official | temp | secondary (If known)', 'name': 'use', 'mergingScript': None, 'isProfile': None, 'type': 'code', 'inputColumns': [], 'attributes': []}, {'id': 'cjpicvbldusnd0a57nwyk4xet', 'comment': 'Description of identifier', 'name': 'type', 'mergingScript': None, 'isProfile': None, 'type': 'CodeableConcept', 'inputColumns': [], 'attributes': [{'id': 'cjpicvblgusnf0a57ktei4ozm', 'comment': 'Code defined by a terminology system', 'name': 'coding', 'mergingScr